In [ ]:
import warnings

warnings.filterwarnings("ignore")

# Induced gap and magnetic field

In [ ]:
# 1. Standard library imports
import asyncio
from copy import copy
from functools import partial
from operator import itemgetter
from pathlib import Path

# 2. External package imports
import adaptive
import holoviews as hv
import kwant
import matplotlib.pyplot as plt
import numpy as np

# 3. Internal imports
import common
import funcs
from common import loss

# hv.notebook_extension('matplotlib')
print(kwant.__version__)

adaptive.notebook_extension()

# Connect to ipyparallel

In [ ]:
# import hpc05
# hpc05.connect.kill_ipcluster()
# client, dview, lview = hpc05.start_and_connect(99, profile='pbs2',
#                                                folder='~/Work/induced_gap_B_field/',
#                                                timeout=300)

In [ ]:
import hpc05

hpc05.kill_remote_ipcluster()
client, dview, lview = hpc05.start_remote_and_connect(
    300, profile="pbs", folder="~/Work/induced_gap_B_field/", timeout=300
)
# client, dview, lview = hpc05.connect_ipcluster(300, profile='pbs', folder='~/Work/induced_gap_B_field/')

# Usage

In [ ]:
syst_pars = dict(
    a=10,
    angle=0,
    onsite_disorder=False,
    L=200,
    coverage_angle=135,
    r1=35,
    r2=70,
    shape="circle",
    with_leads=True,
    with_shell=True,
    A_correction=True,
)

params = dict(
    alpha=20,
    B_x=0.5,
    B_y=0,
    B_z=0,
    Delta=60,
    g=50,
    orbital=True,
    mu_sc=100,
    mu=7,
    mu_lead=7,
    c_tunnel=3 / 4,
    V=lambda x, y, z: 0,
    **funcs.constants.__dict__
)

# syst_pars['L_barrier'] = 100
# params['V_barrier_mu'] = syst_pars['L_barrier']/2
# params['V_barrier_sigma'] = syst_pars['L_barrier']/10
# params['V_barrier_height'] = 30
# params['V_barrier'] = funcs.V_barrier

syst_pars["L_barrier"] = 10

syst = funcs.make_3d_wire(**syst_pars)

kwant.plot(syst)

In [ ]:
lead = syst.leads[0]
funcs.plot_wfs_in_cross_section(lead, params, 0)

In [ ]:
%%opts Image [colorbar=True] (cmap='RdBu_r')

x = 20  # Take cross-section at
mu = 15  # meV
xy = funcs.get_cross_section(syst, x, 0)
ims = {}
for grad in range(-10, 6, 2):
    V = lambda x, y, z: grad * z / syst_pars["r1"]

    potential = np.array([mu - V(*s.pos) for s in syst.sites if s.pos[0] == x])
    mus = np.rot90(kwant.plotter.mask_interpolate(xy, potential, oversampling=1)[0])
    ims[grad] = hv.Image(mus, label="Chemical potential")
hv.HoloMap(ims)

In [ ]:
# %%opts Path [aspect='square']
params = dict(
    c_tunnel=3 / 4,
    B_x=0.0,
    B_y=0,
    B_z=0,
    V_barrier=40,
    g=50,
    mu_sc=100,
    alpha=20,
    orbital=False,
    V=lambda x, y, z: 10 * z / 35,
    Delta=60,
    **funcs.constants.__dict__
)

lead_pars = dict(
    a=10,
    angle=0,
    coverage_angle=135,
    r1=35,
    r2=70,
    shape="circle",
    with_shell=True,
    A_correction=True,
    rotate_spin_orbit=False,
)

lead = funcs.make_lead(**lead_pars).finalized()
params["mu_lead"] = params["mu_sc"] = 0
params["B_y"] = 0
ks = np.linspace(-1, 1, 101)
Es = funcs.bands(lead, params, ks)
p1 = hv.Path((ks, Es))[:, -100:100]

p1[:, -25:25]  # + p1[:, -2:2]

In [ ]:
funcs.find_gap(lead, params)

In [ ]:
%%time
gap = funcs.find_gap(lead, params)
print(f"The bandgap is {gap} meV")

In [ ]:
params = dict(
    alpha=20,
    B_x=0,
    B_y=0,
    B_z=0,
    Delta=60,
    g=50,
    orbital=True,
    mu=15,
    mu_lead=15,
    mu_sc=100,
    c_tunnel=3 / 4,
    V=lambda x, y, z: 0,
    V_barrier=50,
    **funcs.constants.__dict__
)

S = kwant.smatrix(syst, params=params)

In [ ]:
funcs.andreev_conductance(syst, params, 0)

# Tuning the gap

In [ ]:
params = dict(
    c_tunnel=3 / 4,
    B_x=0,
    B_y=0,
    B_z=0,
    g=50,
    alpha=20,
    orbital=True,
    V="lambda x, y, z: 0 * z / 35",
    mu_sc=100,
    **funcs.constants.__dict__
)

syst_pars = dict(
    a=10,
    angle=0,
    coverage_angle=135,
    r1=35,
    r2=70,
    shape="circle",
    with_shell=True,
    A_correction=True,
    rotate_spin_orbit=False,
)


def lowest_energy(x, syst_pars, params):
    import funcs
    import numpy as np

    lead = funcs.make_lead(**syst_pars).finalized()
    params["mu_lead"], params["Delta"] = x
    # Combine the fixed parameters `params` and the changing
    # parameters `val` to one dict and evaluate the string
    # lambda functions.
    params = common.parse_params(params)

    # Create the Hamiltonian `ham` at k=0.
    h0 = lead.cell_hamiltonian(params=params)
    t0 = lead.inter_cell_hopping(params=params)
    ham = h0 + t0 + t0.conj().T

    # Find the energies.
    ev = np.linalg.eigvalsh(ham)

    # Return a combined dictionary with the results and input.
    return np.abs(ev).min()


def find_crossings(f, g):
    return np.argwhere(np.diff(np.sign(f - g)) != 0).reshape(-1)


def get_Delta(learner, Delta_ind):
    plot = learner.plot(n=1000).Image.I.redim(x="mu", y="Delta", z="E_gap")
    line = plot.reduce(["mu"], np.min)
    idx = find_crossings(line.data["E_gap"], Delta_ind)[-1]
    Delta = line.data["Delta"][idx]
    return Delta

In [ ]:
learner = adaptive.Learner2D(
    partial(lowest_energy, syst_pars=syst_pars, params=params),
    [(5, 20), (0, 200)],
    loss,
)

runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.001)

In [ ]:
runner.task.print_stack()

In [ ]:
plot = learner.plot().Image.I.redim(x="mu", y="Delta", z="E_gap")
plot

In [ ]:
%%opts Scatter (size=10, color='r')
Delta_ind = 0.4

line = plot.reduce(["mu"], np.min)
line * hv.HLine(0.4) * hv.Scatter((get_Delta(learner, Delta_ind), Delta_ind))

In [ ]:
Deltas = np.arange(*learner.bounds[1])
hm = hv.HoloMap({Delta: plot.sample(Delta=Delta)[:, 0:1] for Delta in Deltas})
hm * hv.HLine(0.4)

#### Run it for different potentials

In [ ]:
import itertools

params = dict(
    c_tunnel=3 / 4,
    B_x=0,
    B_y=0,
    B_z=0,
    g=50,
    alpha=20,
    orbital=True,
    mu_sc=100,
    **funcs.constants.__dict__,
)

syst_pars = dict(
    a=10,
    angle=None,
    coverage_angle=135,
    r1=35,
    r2=70,
    shape="circle",
    with_shell=True,
    A_correction=True,
    rotate_spin_orbit=False,
)

learners = []
gradients = np.arange(-10, 8, 2)
angles = [0, 45]

vals = adaptive.utils.named_product(angle=angles, grad=gradients)

for val in vals:
    syst_pars["angle"] = val["angle"]
    gradient = val["grad"]
    r1 = syst_pars["r1"]
    _theta_V = np.deg2rad(90)
    coord = f"np.sin({_theta_V}) * z + np.cos({_theta_V}) * y"
    params["V"] = f"lambda x, y, z: {gradient} * ({coord}) / {r1}"

    f = partial(lowest_energy, syst_pars=copy(syst_pars), params=copy(params))
    learner = adaptive.Learner2D(f, [(5, 20), (0, 250)], loss)
    learners.append(learner)

learner = adaptive.BalancingLearner(learners)

In [ ]:
runner = adaptive.Runner(learner, executor=client, goal=lambda l: l.loss() < 0.0001)

In [ ]:
runner.task.print_stack(), learner.loss(), sum(
    l.npoints for l in learner.learners
) / len(learner.learners)

In [ ]:
learner.plot(plotter=lambda l: l.plot().redim(x="mu", y="Delta", z="E_gap"))

In [ ]:
import collections

Delta_ind = 0.4
plots = {}
gaps = collections.defaultdict(dict)
for cdims, l in zip(vals, learner.learners):
    plot = l.plot(n=1000).Image.I.redim(x="mu", y="Delta", z="E_gap")
    line = plot.reduce(["mu"], np.min)
    try:
        Delta = get_Delta(l, Delta_ind)
    except IndexError:
        Delta = np.nan
    plots[tuple(cdims.values())] = line * hv.HLine(0.4) * hv.Scatter((Delta, Delta_ind))
    gaps[cdims["grad"]][cdims["angle"]] = Delta

In [ ]:
dict(gaps)  # Use these in the "Sweep electric field gradient" simulation below.

In [ ]:
%%opts Scatter (size=5 color='r')
hv.HoloMap(plots, kdims=["angle", "gradient"])

In [ ]:
runner.cancel()

# conductance $V_{bias}$ vs $B_x$

In [ ]:
def conductance(x, val, syst_pars, params):
    import funcs
    import common

    for k in ["angle"]:
        try:
            syst_pars[k] = val[k]
        except KeyError:
            pass

    params = common.parse_params(dict(**params, **val))

    params["mu_lead"] = params["mu"]
    params["B_x"], params["B_y"], params["B_z"] = (0, 0, 0)
    val["V_bias"], params["B_{}".format(val["direction"])] = x

    syst = funcs.make_3d_wire(**syst_pars)
    return funcs.conductance(syst, params, E=val["V_bias"])

### Sweep electric field gradient

In [ ]:
gaps = {
    -10: {0: 193.625, 45: 113.125},
    -8: {0: 165.875, 45: 96.625},
    -6: {0: 147.375, 45: 79.125},
    -4: {0: 119.875, 45: 63.125},
    -2: {0: 98.625, 45: 46.625},
    0: {0: 78.125, 45: 32.125},
    2: {0: 59.875, 45: 20.875},
    4: {0: 39.375, 45: 13.125},
}

params = dict(
    c_tunnel=3 / 4, V_barrier=40, mu_sc=100, mu=15, **funcs.constants.__dict__
)

syst_pars = dict(
    a=10,
    onsite_disorder=False,
    L=2000,
    coverage_angle=135,
    r1=35,
    r2=70,
    shape="circle",
    with_leads=True,
    with_shell=True,
    A_correction=True,
)

vals = adaptive.utils.named_product(
    g=[0, 50],
    alpha=[0, 20],
    orbital=[False, True],
    direction=list("xyz"),
    gradient=list(gaps.keys()),
    angle=[0, 45],
)

learners = []
for val in vals:
    params["V"] = f'lambda x, y, z: {val["gradient"]} * z / {syst_pars["r1"]}'
    params["Delta"] = gaps[val["gradient"]][val["angle"]]
    f = partial(conductance, val=val, params=copy(params), syst_pars=copy(syst_pars))
    learner = adaptive.Learner2D(f, [(-1, 1), (0, 2)], loss)
    learners.append(learner)

learner = adaptive.BalancingLearner(learners)
folder = Path("data/gradient-sweep-angle-0-45")
fnames = [folder / f"data_learner_{i:04d}.pickle" for i in range(len(learners))]
learner.load(fnames)

### Change alpha

In [ ]:
gaps = {
    -10: {0: 193.625, 45: 113.125},
    -8: {0: 165.875, 45: 96.625},
    -6: {0: 147.375, 45: 79.125},
    -4: {0: 119.875, 45: 63.125},
    -2: {0: 98.625, 45: 46.625},
    0: {0: 78.125, 45: 32.125},
    2: {0: 59.875, 45: 20.875},
    4: {0: 39.375, 45: 13.125},
}

params = dict(c_tunnel=3 / 4, V_barrier=40, mu_sc=100, **funcs.constants.__dict__)

syst_pars = dict(
    a=10,
    onsite_disorder=False,
    L=2000,
    coverage_angle=135,
    r1=35,
    r2=70,
    shape="circle",
    with_leads=True,
    with_shell=True,
    A_correction=True,
)

vals = adaptive.utils.named_product(
    g=[0, 50],
    alpha=[0, 20, 50, 100, 250],
    orbital=[False, True],
    direction=list("xy"),
    gradient=list(gaps.keys()),
    mu=[10, 12, 15],
    angle=[0],
)

learners = []
for val in vals:
    params["V"] = f'lambda x, y, z: {val["gradient"]} * z / {syst_pars["r1"]}'
    params["Delta"] = gaps[val["gradient"]][val["angle"]]
    f = partial(conductance, val=val, params=copy(params), syst_pars=copy(syst_pars))
    learner = adaptive.Learner2D(f, [(-1, 1), (0, 2)], loss)
    learners.append(learner)

learner = adaptive.BalancingLearner(learners)
folder = Path("data/gradient-sweep-alpha2")
fnames = [folder / f"data_learner_{i:04d}.pickle" for i in range(len(learners))]
learner.load(fnames)

### Sweep chemical potential

In [ ]:
syst_pars = dict(
    a=10,
    angle=0,
    onsite_disorder=False,
    L=2000,
    coverage_angle=135,
    r1=35,
    r2=70,
    shape="circle",
    with_leads=True,
    with_shell=True,
    A_correction=True,
)

params = dict(
    Delta=78.125,
    c_tunnel=3 / 4,
    V="lambda x, y, z: 0",
    mu_sc=100,
    V_barrier=40,
    **funcs.constants.__dict__
)

vals = adaptive.utils.named_product(
    g=[0, 50],
    alpha=[0, 20],
    orbital=[False, True],
    direction=list("xyz"),
    mu=list(range(10, 21)),
)

learners = []
for val in vals:
    f = partial(conductance, val=val, params=params, syst_pars=syst_pars)
    learner = adaptive.Learner2D(f, [(-1, 1), (0, 2)], loss)
    learners.append(learner)


learner = adaptive.BalancingLearner(learners, cdims=vals)
folder = Path("data/mu-sweep2")
fnames = [folder / f"data_learner_{i:04d}.pickle" for i in range(len(learners))]
learner.load(fnames)

### Rotation

In [ ]:
def conductance_rotation(x, val, syst_pars, params):
    import funcs, common

    for k in ["angle"]:
        syst_pars[k] = val[k]

    params["mu_lead"] = params["mu"]
    val["V_bias"], theta = x
    val["B_x"], val["B_y"], val["B_z"] = common.spherical_coords(params["B"], theta, 90)

    params = common.parse_params(dict(**params, **val))

    syst = funcs.make_3d_wire(**syst_pars)
    return funcs.conductance(syst, params, E=val["V_bias"])


gaps = {
    -10: {0: 193.625, 45: 113.125},
    -8: {0: 165.875, 45: 96.625},
    -6: {0: 147.375, 45: 79.125},
    -4: {0: 119.875, 45: 63.125},
    -2: {0: 98.625, 45: 46.625},
    0: {0: 78.125, 45: 32.125},
    2: {0: 59.875, 45: 20.875},
    4: {0: 39.375, 45: 13.125},
}

params = dict(
    c_tunnel=3 / 4,
    V_barrier=40,
    mu_sc=100,
    mu=15,
    B=0.25,
    sin="lambda x: np.sin(np.deg2rad(x))",
    cos="lambda x: np.cos(np.deg2rad(x))",
    **funcs.constants.__dict__,
)

syst_pars = dict(
    a=10,
    onsite_disorder=False,
    L=2000,
    coverage_angle=135,
    r1=35,
    r2=70,
    shape="circle",
    with_leads=True,
    with_shell=True,
    A_correction=True,
    rotate_spin_orbit=True,
)

vals = adaptive.utils.named_product(
    g=[0, 50],
    alpha=[0, 20],
    orbital=[True, False],
    gradient=list(gaps.keys()),
    angle=[0, 45],
    theta_SO=np.linspace(0, 90, 7),
    theta_V=[90],
)

learners = []
for val in vals:
    gradient = val["gradient"]
    r = syst_pars["r1"]
    theta_V = np.deg2rad(val["theta_V"])
    coord = f"np.sin({theta_V}) * z + np.cos({theta_V}) * y"
    params["V"] = f"lambda x, y, z: {gradient} * ({coord}) / {r}"
    params["Delta"] = gaps[gradient][val["angle"]]

    f = partial(
        conductance_rotation, val=val, params=copy(params), syst_pars=copy(syst_pars)
    )
    learner = adaptive.Learner2D(f, [(-1, 1), (-90, 135)], loss)
    learners.append(learner)

learner = adaptive.BalancingLearner(learners, cdims=vals)
folder = Path("data/gradient-sweep-rotation-0-90-move-SO-slowly")
fnames = [folder / f"data_learner_{i:04d}.pickle" for i in range(len(learners))]
learner.load(fnames)

In [ ]:
%%output size=200
lead_pars = dict(
    a=10,
    angle=45,
    coverage_angle=135,
    r1=35,
    r2=70,
    shape="circle",
    with_shell=True,
    A_correction=True,
    rotate_spin_orbit=True,
)

lead = funcs.make_lead(**lead_pars).finalized()

params = dict(
    c_tunnel=3 / 4,
    mu_sc=100,
    mu=15,
    B=0.25,
    sin="lambda x: np.sin(np.deg2rad(x))",
    cos="lambda x: np.cos(np.deg2rad(x))",
    **funcs.constants.__dict__,
    g=50,
    alpha=20,
    orbital=True,
    gradient=4,
    theta_SO=45,
    theta_V=45,
)

gradient = params["gradient"]
r = syst_pars["r1"]
theta_V = np.deg2rad(params["theta_V"])
coord = f"np.sin({theta_V}) * z + np.cos({theta_V}) * y"
params["V"] = f"lambda x, y, z: {gradient} * ({coord}) / {r}"
params["Delta"] = gaps[gradient][lead_pars["angle"]]
params["mu_lead"] = params["mu_sc"] = 15
params = common.parse_params(params)

plots = {}
for theta in np.linspace(0, 90, 11):

    params["B_x"], params["B_y"], params["B_z"] = common.spherical_coords(
        params["B"], theta, 90
    )

    ks = np.linspace(-3, 3, 201)
    Es = funcs.bands(lead, params, ks)
    p1 = hv.Path((ks, Es))[:, -100:100]
    plots[theta] = p1
hv.HoloMap(plots)

## Run only selected learners

In [ ]:
all_learners, all_fnames = [], []

In [ ]:
fnames_new, learners_new = zip(*[(f, l) for l, f in zip(learners, fnames) if l.data])
all_learners += learners_new
all_fnames += fnames_new

In [ ]:
import hpc05
client, dview, lview = hpc05.start_remote_and_connect(
    120, profile="pbs", folder="~/spin-orbit-nanowires/", timeout=300
)

In [ ]:
new_learner = adaptive.BalancingLearner(all_learners, strategy="npoints")
runner = adaptive.Runner(new_learner, retries=10, executor=client)
task = runner.start_periodic_saving(dict(fname=all_fnames), 120)
runner.live_info()

In [ ]:
new_learner.save(fnames_new)

In [ ]:
runner.task.print_stack()

In [ ]:
sorted([(l.npoints, f) for f, l in zip(all_fnames, new_learner.learners)])

In [ ]:
n, f, l = min([(l.npoints, f, l) for f, l in zip(all_fnames, new_learner.learners)], key=lambda x: x[0])
n, f, l

In [ ]:
%%output size=200
l.plot(n=400, tri_alpha=0.2)

## Run

In [ ]:
runner = adaptive.Runner(learner, executor=client, shutdown_executor=False, log=False)
learner.start_periodic_saver(runner, folder, "data_learner_{}.pickle", interval=300)

In [ ]:
learner.save(folder, "data_learner_{}.pickle")

In [ ]:
# runner.task.cancel()
# backup.cancel()
runner.task.print_stack()

In [ ]:
sum(l.npoints for l in learners) / len(learners)

In [ ]:
idx = np.argmin([l.n for l in learner.learners])
learner._points[idx][-1][0] = np.inf
idx, learners[idx].n

In [ ]:
%%output size=150
learner.learners[idx].plot(tri_alpha=0)

In [ ]:
%%opts Image {+framewise} [colorbar=True]
%%output size=200

learner.plot(cdims=vals)